# Deep Learning with PyTorch: A 60 Minute Blitz

This is me following along to the [tutorial from the official PyTorch website](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py). Note that I'm also taking some asides and detours as I type the code to investigate interesting detours as I seek to more fully understand the runtime.

The first thing that we need to do is import the torch library. This takes a fair amount of time. Fortunately, this is a one-time operation per session.

In [1]:
import torch

This Hello, World example initializes a new tensor of dimensions `[5,3]` that contain random values from whatever the memory was before the call, i.e., it does not zero it out.

In [2]:
x = torch.empty(5, 3)
x

tensor([[-1.2499e+14,  4.5799e-41, -1.2499e+14],
        [ 4.5799e-41,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.3378e-15],
        [ 3.0873e-41,  7.1547e+22,  4.5828e+30],
        [ 4.4842e-44,  0.0000e+00,         nan]])

This next example constructs a tensor, but this time initialized to zeros:

In [3]:
x = torch.zeros(5, 3)
x

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

You can also specify the _type_ of the zeros that you want, e.g., half precision floating point (which is interesting for its performance characteristics)

In [4]:
x = torch.zeros(5, 3, dtype=torch.half)
x

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float16)

You can also create it with explicit random values:

In [5]:
x = torch.rand(5, 3, dtype=torch.float)
x

tensor([[0.0955, 0.5926, 0.1150],
        [0.6357, 0.3031, 0.2330],
        [0.9922, 0.4786, 0.3525],
        [0.1751, 0.2733, 0.5053],
        [0.4166, 0.9786, 0.9742]])

But if you try and use a type that is only supported on GPUs, e.g., `torch.half`, it will throw an error because `torch.rand` by default will construct the type on the CPU:

In [6]:
x = torch.rand(5, 3, dtype=torch.half)
x

RuntimeError: _th_uniform_ not supported on CPUType for Half

So we solve this by first initializing a `cuda` device:

In [7]:
device = torch.device("cuda")
device

device(type='cuda')

We can pass the `cuda` device as a parameter to `torch.rand` which now constructs the tensor on the GPU successfully. This requires 1 time initialization of the CUDA infrastructure (several seconds), again a per-session cost.

In [8]:
x = torch.rand(5, 3, device=device, dtype=torch.half)
x

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

The size of a tensor is sometimes something that we need to know when debugging. We can do this by using the `size()` member function:

In [9]:
print(x.size())

torch.Size([5, 3])


Tensors support operations such as addition, which does a memberwise addition of all of the elements across tensors of the same size. Let's create a new tensor, y, of the same size with random values:

In [10]:
y = torch.rand(5, 3, device=device, dtype=torch.half)
y

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

We can do an implicit add simply by using the plus operator:

In [ ]:
print(x+y)

Or we can explicitly use the `torch.add()` function:

In [ ]:
print(torch.add(x, y))

What happens if the tensor sizes are mismatched? Let's find out:

In [ ]:
z = torch.rand(2, 2, device=device, dtype=torch.half)
z

As predicted, we get a runtime error when attempting the operation:

In [ ]:
print(x+z)

In the cases above, we've been adding numbers and producing a new tensor. We can also do in-place addition as well which will replace the value in the tensor for which the `add_()` member is being invoked on:

In [ ]:
y.add_(x)
y

What happens when we add a scalar to a tensor? Pretty much what you would expect, the scalar gets added to each member of the tensor:

In [ ]:
print(y+1)

## Tensors and CUDA

Tensors can be moved back and forth between CPU and GPU as well. You can use the `to()` method on the tensor to move a tensor back to the CPU.

In [ ]:
cpu_y = y.to("cpu", torch.double)

In [ ]:
cpu_y

## Automatic Differentiation

This is the key feature of pytorch (and other deep learning packages) which provides automatic differentiation for all operations on tensors. You can turn on automatic differentiation by setting the `requires_grad=True` on the tensor:

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
x

In [ ]:
y = x + 2
y

In [ ]:
z = y * 2
z

## Neural networks

Let's build a neural network using pytorch by first importing 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Next, let's define a convolutional neural network.

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
net

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/test1')

In [ ]:
import torchvision
import torchvision.transforms as transforms
# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])
# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

In [ ]:
import matplotlib.pyplot as plt

# helper function to show an image
# (used in the `plot_classes_preds` function below)
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        
# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# show images
matplotlib_imshow(img_grid, one_channel=True)

# write to tensorboard
writer.add_image('four_fashion_mnist_images', img_grid)